## High Resolution Image generation Code for Dr.Sabato's Honeywell Proposal

### For FFN Model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from keras.models import Sequential, load_model
from keras import layers
import keras
import tensorflow as tf
import os
import time

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Create output directory for high-resolution images
output_folder = "high_res_ffn_images"
os.makedirs(output_folder, exist_ok=True)

# Part 1: Train and save the FFN model
def train_and_save_model():
    print("Loading training and test data...")
    Train_df = pd.read_csv(r"C:\Users\G_Modak\Desktop\Honeywell Project\FEMap Models\FAZ_FEModel Files\Thermal analysis_temp on each nodes_different thickness inserts_cropped_100x100_please1.csv")
    Test_df = pd.read_csv(r"C:\Users\G_Modak\Desktop\Honeywell Project\FEMap Models\FAZ_FEModel Files\Thermal analysis_temp on each nodes_different thickness inserts_cropped_100x100_please1.csv")

    # Normalize the data
    max_temp = 333.0
    Train = pd.DataFrame(Train_df).to_numpy() / max_temp
    Test = pd.DataFrame(Test_df).to_numpy() / max_temp

    # Create sparse mask with 5% of the data
    mask_ratio = 0.05
    np.random.seed(42)  # For reproducibility
    mask = np.random.binomial(1, mask_ratio, Train[0,:].shape)

    # Create sparse input data
    Train_masked = Train * mask
    Test_masked = Test * mask

    # Build the autoencoder model
    input_data = keras.Input(shape=(10000,))
    encoded = layers.Dense(128, activation='relu')(input_data)
    encoded = layers.Dense(64, activation='relu')(encoded)
    encoded = layers.Dense(32, activation='relu')(encoded)
    decoded = layers.Dense(64, activation='relu')(encoded)
    decoded = layers.Dense(128, activation='relu')(decoded)
    decoded = layers.Dense(10000, activation='sigmoid')(decoded)

    # Create and compile the autoencoder
    autoencoder = keras.Model(input_data, decoded)
    autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

    # Train the autoencoder
    print("Training the FFN autoencoder model...")
    history = autoencoder.fit(
        Train_masked, Train,
        epochs=100,
        batch_size=4,
        shuffle=True,
        validation_data=(Test_masked, Test),
        verbose=1
    )

    # Save the model
    model_filename = f'ffn_autoencoder_{mask_ratio*100}percent.keras'
    autoencoder.save(model_filename)
    print(f"Model saved as {model_filename}")

    # Plot and save loss history
    plt.figure(figsize=(10, 6))
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('FFN Autoencoder Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    loss_plot_filename = f'autoencoder_loss_FFN_{mask_ratio*100}%.png'
    plt.savefig(loss_plot_filename)
    plt.close()
    print(f"Loss plot saved as {loss_plot_filename}")

    return model_filename, mask_ratio, max_temp

# Part 2: Generate high-resolution images at every 100th timestep
def generate_high_resolution_images(model_filename, mask_ratio, max_temp, vmin=295.0, vmax=335.0, dpi=600):
    print("Loading data for image generation...")
    # Load the data
    df = pd.read_csv(r"C:\Users\G_Modak\Desktop\Honeywell Project\FEMap Models\FAZ_FEModel Files\Thermal analysis_temp on each nodes_different thickness inserts_cropped_100x100_please1.csv")
    
    # Load the model
    print(f"Loading trained model from {model_filename}...")
    autoencoder = load_model(model_filename)
    
    # Normalize the data
    data = pd.DataFrame(df).to_numpy() / max_temp
    
    # Create sparse mask with same ratio as training
    np.random.seed(42)  # For reproducibility - use same seed as training
    mask = np.random.binomial(1, mask_ratio, data[0,:].shape)
    
    # Create sparse input data
    data_masked = data * mask
    
    # Generate predictions
    print("Generating predictions with the model...")
    predicted_data = autoencoder.predict(data_masked)
    
    # Define timesteps to use
    # timesteps = range(900, 1000, 100)  # Every 100th timestep from 0 to 900
    timesteps = [0,100,200,300,400,500,600,700,800,900,999]
    
    # Set grid dimensions
    grid_size = 100  # 100x100 grid
    
    # Process each selected timestep
    for timestep in timesteps:
        print(f"Generating high-resolution images for timestep {timestep}...")
        
        # Get original, masked, and reconstructed data for this timestep
        original = data[timestep].reshape(grid_size, grid_size) * max_temp
        masked = data_masked[timestep].reshape(grid_size, grid_size) * max_temp
        reconstructed = predicted_data[timestep].reshape(grid_size, grid_size) * max_temp
        
        # Calculate relative error
        with np.errstate(divide='ignore', invalid='ignore'):
            relative_error = (original - reconstructed) / original
            # Handle division by zero or very small values
            relative_error = np.where(np.isfinite(relative_error), relative_error, 0)
        
        # Calculate error bounds for better visualization
        p05 = np.percentile(relative_error.flatten(), 5)
        p95 = np.percentile(relative_error.flatten(), 95)
        error_max = max(abs(p05), abs(p95))
        
        # Create a 2x2 figure for all four images
        fig, axes = plt.subplots(2, 2, figsize=(20, 16), dpi=dpi)
        axes = axes.flatten()
        
        # Set titles for each subplot
        titles = ["Original Temperature Field", 
                f"Sparse Data ({mask_ratio*100}%)",
                "FFN Reconstructed Field", 
                "Relative Error ((Original-Reconstructed)/Original)"]
        
        # Plot each image
        im0 = axes[0].imshow(original, cmap='viridis', vmin=vmin, vmax=vmax)
        im1 = axes[1].imshow(masked, cmap='viridis', vmin=vmin, vmax=vmax)
        im2 = axes[2].imshow(reconstructed, cmap='viridis', vmin=vmin, vmax=vmax)
        im3 = axes[3].imshow(relative_error, cmap='viridis', vmin=-error_max, vmax=error_max)
        
        # Add colorbars
        cbar0 = fig.colorbar(im0, ax=axes[0])
        cbar0.set_label('Temperature (K)', fontsize=12)
        cbar1 = fig.colorbar(im1, ax=axes[1])
        cbar1.set_label('Temperature (K)', fontsize=12)
        cbar2 = fig.colorbar(im2, ax=axes[2])
        cbar2.set_label('Temperature (K)', fontsize=12)
        cbar3 = fig.colorbar(im3, ax=axes[3])
        cbar3.set_label('Relative Temperature Difference', fontsize=12)
        
        # Set titles and labels
        for i, title in enumerate(titles):
            axes[i].set_title(title, fontsize=14)
            axes[i].set_xlabel('Space (x)', fontsize=12)
            axes[i].set_ylabel('Space (y)', fontsize=12)
        
        # Add main title
        plt.suptitle(f"FFN Thermal Field Reconstruction (Timestep: {timestep})", fontsize=16)
        
        # Adjust spacing
        plt.tight_layout(rect=[0, 0, 1, 0.96])
        
        # Save combined image
        combined_file_path = os.path.join(output_folder, f"ffn_combined_view_timestep_{timestep}.png")
        plt.savefig(combined_file_path, dpi=dpi, bbox_inches='tight')
        plt.close()
        
        print(f"Saved combined image to {combined_file_path}")
        
        # Now generate individual high-resolution images for each component
        components = {
            'original': (original, 'Original Temperature Field', vmin, vmax, 'Temperature (K)'),
            'sparse': (masked, f'Sparse Data ({mask_ratio*100}%)', vmin, vmax, 'Temperature (K)'),
            'reconstructed': (reconstructed, 'FFN Reconstructed Field', vmin, vmax, 'Temperature (K)'),
            'relative_error': (relative_error, 'Relative Error ((Original-Reconstructed)/Original)', 
                            -error_max, error_max, 'Relative Temperature Difference')
        }
        
        for comp_name, (data_array, title, v_min, v_max, cbar_label) in components.items():
            # Create figure
            plt.figure(figsize=(10, 8), dpi=dpi)
            
            # Plot image
            im = plt.imshow(data_array, cmap='viridis', vmin=v_min, vmax=v_max)
            
            # Add colorbar
            cbar = plt.colorbar(im)
            cbar.set_label(cbar_label, fontsize=12)
            
            # Add title and labels
            plt.title(title, fontsize=14)
            plt.xlabel('Space (x)', fontsize=12)
            plt.ylabel('Space (y)', fontsize=12)
            
            # Tight layout
            plt.tight_layout()
            
            # Save image
            file_path = os.path.join(output_folder, f"ffn_{comp_name}_timestep_{timestep}.png")
            plt.savefig(file_path, dpi=dpi, bbox_inches='tight')
            plt.close()
            
            print(f"Saved {comp_name} image to {file_path}")

    print(f"All high-resolution images saved in folder: {output_folder}")

# Main execution
if __name__ == "__main__":
    start_time = time.time()
    
    # First train and save the model
    # model_filename, mask_ratio, max_temp = train_and_save_model()
    
    # Then generate high-resolution images
    generate_high_resolution_images(model_filename, mask_ratio, max_temp, vmin=295.0, vmax=335.0, dpi=600)
    
    end_time = time.time()
    elapsed_time = end_time - start_time
    print(f"Total execution time: {elapsed_time:.2f} seconds ({elapsed_time/60:.2f} minutes)")

### For CNN Model

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import Normalize
import tensorflow as tf
from tensorflow.keras.models import load_model
import os

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Define grid dimensions
grid_size = 100  # 100x100 grid

# Define mask ratio parameter
mask_ratio = 0.05  # 5% of data

# Path configurations
model_path = f'thermal_reconstruction_CNN_{mask_ratio}%.keras'
data_path = r"C:\Users\G_Modak\Desktop\Honeywell Project\FEMap Models\FAZ_FEModel Files\Thermal analysis_temp on each nodes_different thickness inserts_cropped_100x100_please1.csv"
output_folder = "high_res_thermal_images"

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Load the trained model
try:
    autoencoder = load_model(model_path, compile=False)
    print(f"Model loaded successfully from {model_path}")
except Exception as e:
    print(f"Couldn't load the model from {model_path}: {e}")
    print("Make sure to run the original training script first or adjust the model path.")
    exit()

# Load the data
try:
    df = pd.read_csv(data_path)
    print(f"Data loaded successfully from {data_path}")
except Exception as e:
    print(f"Couldn't load the data from {data_path}: {e}")
    print("Please adjust the data path to point to your CSV file.")
    exit()

# Extract the data values
data = df.values

# Get normalization parameters
data_min = np.min(data)
data_max = np.max(data)
print(f"Temperature range: {data_min:.2f} to {data_max:.2f}")

# Create a fixed mask for all samples (same as in training)
fixed_mask = np.random.binomial(1, mask_ratio, (grid_size, grid_size))

def prepare_data_for_timestamp(timestamp_idx):
    """Extract and prepare data for a specific timestamp"""
    if timestamp_idx >= len(data):
        print(f"Warning: Timestamp {timestamp_idx} exceeds available data. Using modulo.")
        timestamp_idx = timestamp_idx % len(data)
        
    # Extract temperature field for the timestamp
    temp_field = data[timestamp_idx].reshape(grid_size, grid_size)
    
    # Normalize the data
    temp_field_norm = (temp_field - data_min) / (data_max - data_min)
    
    # Apply the mask to create sparse input
    temp_field_masked = temp_field_norm * fixed_mask
    
    # Reshape for model input
    model_input = temp_field_masked.reshape(1, grid_size, grid_size, 1)
    
    return temp_field, temp_field_masked.reshape(grid_size, grid_size), model_input

def reconstruct_thermal_field(model, model_input):
    """Reconstruct a full thermal field from sparse measurements"""
    # Make prediction
    prediction = model.predict(model_input, verbose=0)
    
    # Extract and denormalize
    reconstructed_norm = prediction[0, :, :, 0]
    reconstructed = reconstructed_norm * (data_max - data_min) + data_min
    
    return reconstructed

def generate_high_resolution_images(timestamp_idx, vmin=295.0, vmax=335.0, dpi=600):
    """Generate high-resolution images for a given timestamp"""
    print(f"Generating high-resolution images for timestamp {timestamp_idx}...")
    
    # Prepare data for the timestamp
    original, masked_normalized, model_input = prepare_data_for_timestamp(timestamp_idx)
    
    # Denormalize the masked data for visualization
    masked = masked_normalized * (data_max - data_min) + data_min
    
    # Reconstruct the thermal field
    reconstructed = reconstruct_thermal_field(autoencoder, model_input)
    
    # Calculate relative error
    with np.errstate(divide='ignore', invalid='ignore'):
        relative_error = (original - reconstructed) / original
        # Handle division by zero or very small values
        relative_error = np.where(np.isfinite(relative_error), relative_error, 0)
    
    # Calculate relative error bounds for better visualization
    # Use 5th and 95th percentile to avoid extreme outliers
    error_values = relative_error.flatten()
    p05 = np.percentile(error_values, 5)
    p95 = np.percentile(error_values, 95)
    error_max = max(abs(p05), abs(p95))
    
    # Generate individual high-resolution images
    image_types = {
        'original': (original, 'Original Temperature Field', vmin, vmax, 'Temperature (K)'),
        'sparse': (masked, f'Sparse Measurements ({mask_ratio*100}%)', vmin, vmax, 'Temperature (K)'),
        'reconstructed': (reconstructed, 'Reconstructed Temperature Field', vmin, vmax, 'Temperature (K)'),
        'relative_error': (relative_error, 'Relative Error ((Original-Reconstructed)/Original)', 
                          -error_max, error_max, 'Relative Temperature Difference')
    }
    
    file_paths = {}
    
    for img_type, (data_array, title, v_min, v_max, cbar_label) in image_types.items():
        # Create figure with larger size for high resolution
        plt.figure(figsize=(10, 8), dpi=dpi)
        
        # Plot the image with viridis colormap
        if img_type == 'relative_error':
            im = plt.imshow(data_array, cmap='viridis', vmin=v_min, vmax=v_max)
        else:
            im = plt.imshow(data_array, cmap='viridis', vmin=v_min, vmax=v_max)
        
        # Add colorbar
        cbar = plt.colorbar(im)
        cbar.set_label(cbar_label, fontsize=12)
        
        # Add title and labels
        plt.title(title, fontsize=14)
        plt.xlabel('Space (x)', fontsize=12)
        plt.ylabel('Space (y)', fontsize=12)
        
        # Tight layout to maximize image size
        plt.tight_layout()
        
        # Save high-resolution image
        file_path = os.path.join(output_folder, f"{img_type}_timestamp_{timestamp_idx}.png")
        plt.savefig(file_path, dpi=dpi, bbox_inches='tight')
        plt.close()
        
        file_paths[img_type] = file_path
        print(f"Saved {img_type} image to {file_path}")
    
    # Create a combined visualization for comparison
    fig, axes = plt.subplots(2, 2, figsize=(20, 16), dpi=dpi)
    axes = axes.flatten()
    
    # Set titles for each subplot
    titles = ["Original Temperature Field", 
              f"Sparse Measurements ({mask_ratio*100}%)",
              "Reconstructed Temperature Field", 
              "Relative Error ((Original-Reconstructed)/Original)"]
    
    # Plot each image in the subplot
    im0 = axes[0].imshow(original, cmap='viridis', vmin=vmin, vmax=vmax)
    im1 = axes[1].imshow(masked, cmap='viridis', vmin=vmin, vmax=vmax)
    im2 = axes[2].imshow(reconstructed, cmap='viridis', vmin=vmin, vmax=vmax)
    im3 = axes[3].imshow(relative_error, cmap='viridis', vmin=-error_max, vmax=error_max)
    
    # Add colorbars
    cbar0 = fig.colorbar(im0, ax=axes[0])
    cbar0.set_label('Temperature (K)', fontsize=12)
    cbar1 = fig.colorbar(im1, ax=axes[1])
    cbar1.set_label('Temperature (K)', fontsize=12)
    cbar2 = fig.colorbar(im2, ax=axes[2])
    cbar2.set_label('Temperature (K)', fontsize=12)
    cbar3 = fig.colorbar(im3, ax=axes[3])
    cbar3.set_label('Relative Temperature Difference', fontsize=12)
    
    # Set titles
    for i, title in enumerate(titles):
        axes[i].set_title(title, fontsize=14)
        axes[i].set_xlabel('Space (x)', fontsize=12)
        axes[i].set_ylabel('Space (y)', fontsize=12)
    
    # Set the main title
    plt.suptitle(f"Thermal Field Reconstruction (Timestamp: {timestamp_idx})", fontsize=16)
    
    # Adjust spacing between subplots
    plt.tight_layout(rect=[0, 0, 1, 0.96])
    
    # Save the combined figure
    combined_file_path = os.path.join(output_folder, f"combined_view_timestamp_{timestamp_idx}.png")
    plt.savefig(combined_file_path, dpi=dpi, bbox_inches='tight')
    plt.close()
    
    print(f"Saved combined image to {combined_file_path}")
    
    return file_paths, combined_file_path

# Generate high-resolution images for selected timestamps
# You can change these to any timestamps you're interested in
selected_timestamps = [0, 100, 200 ,300,400,500,600,700,800,900,1000]

# Temperature range for visualization
vmin = 295.0
vmax = 335.0

# Generate images for each selected timestamp
for timestamp in selected_timestamps:
    generate_high_resolution_images(timestamp, vmin=vmin, vmax=vmax, dpi=600)

print("High-resolution image generation completed successfully!")